# Data Reduction for JWST NIRSpec IFU mosaiced data 

- Step 1: run raw data through the jwst data reduction pipeline to create a clean mosaiced image of the science and reference target. 

- Step 2: center the science and reference target to each other and create bad pixel masks. 

- Step 3: run Reference Differential Imaging to remove central star (Disk = (Star+Disk) - Ref.Star)

- Step 4: model the disk to determine the best fit model, necessary for throughput correction. 

written by: Sarah Betti, 2024

In [ ]:
from preprocessing import *
from centering import *
from make_disk_mask import *
from PSF_subtraction import *
from TP_diskmodeling_DiffEvo import *

Step 1: 
Run the JWST Pipeline reduction.  This does Stages 1-3 with NSCLEAN! 

In [ ]:
# preprocessing.py
dir_name='/Users/sbetti/Documents/Science/datasets/JWST/betaPic_NIRSpec/raw/beta_pic'
output_dir_name='/Users/sbetti/Documents/Science/datasets/JWST/betaPic_NIRSpec'
    
run_preproceessing(dir_name, output_dir_name, verbose=True)


Step 2: center the science and reference image.  
It will output aligned sci and ref cubes as centering/sci_centered.fits and centering/ref_centered.fits


In [ ]:
# centering.py
dir_name = '/Users/sbetti/Documents/Science/datasets/JWST/betaPic_NIRSpec/data_reduction/'
savepath = '/Users/sbetti/Desktop/'
y_center = 66 
x_center = 66 

new_img_size_x = 132
new_img_size_y = 132
filter_size = 25
channel_longest = 4 # use only the first four becuase of the severe saturation 
sci_file_name = 'sci_newoutput_prism-clear_s3d.fits'
ref_file_name = 'ref_newoutput_prism-clear_s3d.fits'
aligned_sci_cube, aligned_ref_cube_rotated = run_centering(dir_name, sci_file_name, ref_file_name, 
                  x_center, y_center, 
                  new_img_size_x, new_img_size_y, 
                  filter_size, channel_longest)

Step 3: 
making disk masks: 
The disk mask was made by using the ds9 region. 
By using the polygon region shape, I manually drew the mask region and then ran run_make_mask to make the mask. 


In [ ]:
# make_disk_mask.py
path_initial = '/Users/sbetti/Documents/Science/datasets/JWST/betaPic_NIRSpec/data_reduction/'
sci_file_name = 'sci_cube_expand_betapic_aligned.fits'
run_make_mask(dir_name, sci_file_name, plot=False)

Step 4:
After creating the necessary disk masks, we can perform the RDI PSF subtraction. 
This gives RDI reduction.  NO throughput correction.
This is done twice!  Once to get wavelength dependent f_RDI, then take average. 
This will output psf_subtraction/RDI_sci_JWST_NIRSpec.fits

In [ ]:
# PSF_subtraction.py
dir_name = '/Users/sbetti/Desktop/'
inner_mask_radius = 15 # HD 181327
outer_mask_radius = 68
y_center = 66 
x_center = 66
sci_file_name = 'sci_newoutput_prism-clear_s3d.fits'
run_PSFsubtraction(dir_name, inner_mask_radius, outer_mask_radius, x_center, y_center, sci_file_name)

Step 6: Disk model
After creating the necessary disk masks, we can calculate the disk model to determine throughput correction. 
This creates a disk model, inserts it, and minimizes the residuals using differential evolution. 
-This produces an .hdf5 file and .fits file of the best fit cube, residuals, and parameters.
- This is done twice!  Once to get wavelength dependent f_RDI, then take average. Rerun using the average f_RDI.  

The final disk model is disk_modeling/bestfit_cube_f{X.XX}.fits

In [ ]:
dstar= 19.44 # distance to the star in pc
itilt = 89.5 # inclination of your disk in degrees
pixel_scale=0.1 # pixel scale in arcsec/px
posang = -7.22
ain = 0.5
aout = -1.5
a = 120 #semi-major axis
ksi0 = 5.1 # reference scale height at the semi-major axis of the disk
gamma = 0.5 # exponant of the vertical exponential decay
beta = 1

dir_name = '/Users/sbetti/Desktop/'
sci_filename = 'sci_centered.fits'
ref_filename = 'ref_centered.fits'
cube_sci_filename = 'sci_newoutput_prism-clear_s3d.fits'
mask_spike = 'spike_mask.fits'
mask_2D = 'disk_mask_0_1_2D_2.fits'
mask_cube = 'mask_cube.fits'
mask_disk_FoV = 'IFU_align_FoV_extra_spike.fits'
y_center = 66 
x_center = 66

bounds = [(100, 8000),(0.1,0.8)]
run_diskmodeling(dir_name, sci_filename, ref_filename, cube_sci_filename, mask_spike, mask_2D, mask_cube,mask_disk_FoV, 
                    x_center, y_center, bounds, 
                    dstar, itilt, pixel_scale, posang, a, ain, aout, ksi0, gamma, beta)